In [1]:
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import iplot

In [2]:
color1 = "#FAA813"
color2 = "#0A0B0A"
color3 = "#FFFFFF"


def gen_xaxis(title):
    """
    Creates the X Axis layout and title
    """
    xaxis = dict(
        title=title,
        titlefont=dict(
            color='#AAAAAA'
        ),
        showgrid=False,
        color='#AAAAAA',
    )
    return xaxis


def gen_yaxis(title):
    """
    Creates the Y Axis layout and title
    """
    yaxis = dict(
        title=title,
        titlefont=dict(
            color='#AAAAAA'
        ),
        showgrid=False,
        color='#AAAAAA',
    )
    return yaxis


def gen_layout(charttitle, xtitle, ytitle, lmarg, h, annotations=None):
    """
    Creates whole layout, with both axis, annotations, size and margin
    """
    return go.Layout(title=charttitle,
                     height=h,
                     width=800,
                     showlegend=False,
                     xaxis=gen_xaxis(xtitle),
                     yaxis=gen_yaxis(ytitle),
                     annotations=annotations,
                     margin=dict(l=lmarg),
                     )


def gen_annotations(annot):
    """
    Generates annotations to insert in the chart
    """
    if annot is None:
        return []

    annotations = []
    for d in annot:
        annotations.append(dict(xref='paper', x=d['x'], y=d['y'],
                           xanchor='left', yanchor='bottom',
                           text=d['text'],
                           font=dict(size=13,
                           color=d['color']),
            showarrow=False))
    return annotations

# 1. Loading data 

In [3]:
vehicle_rentals_df = pd.read_csv("../data/Vehicle_Rentals.csv")

vehicle_rentals_df.head(), vehicle_rentals_df.dtypes

(   Rental_ID  Vehicle_ID  Location_ID  Start_Date    End_Date
 0          1           1            1  02/05/2023  04/05/2023
 1          2           1            1  05/05/2023  07/05/2023
 2          3           1            1  08/05/2023  09/05/2023
 3          4           1            1  10/05/2023  13/05/2023
 4          5           1            1  14/05/2023  15/05/2023,
 Rental_ID       int64
 Vehicle_ID      int64
 Location_ID     int64
 Start_Date     object
 End_Date       object
 dtype: object)

In [4]:
locations_df = pd.read_csv("../data/Locations.csv", delimiter=";")

locations_df.head(), locations_df.dtypes

(   Location_ID       Location_Name       City Country
 0            1   Jardim Paulista I  São Paulo  Brazil
 1            2  Jardim Paulista II  São Paulo  Brazil
 2            3        Vila Olimpia  São Paulo  Brazil
 3            4             Batel I   Curitiba  Brazil
 4            5            Batel II   Curitiba  Brazil,
 Location_ID       int64
 Location_Name    object
 City             object
 Country          object
 dtype: object)

# 2. Data Quality check

Now Let's take a look into the data and see with we found some inconsistency that we should treat.

## 2.1 Missing Check

In [5]:
print("Dados ausentes em Vehicle_Rentals:\n",
      vehicle_rentals_df.isnull().sum())
print("\nDados ausentes em Locations:\n", locations_df.isnull().sum())

Dados ausentes em Vehicle_Rentals:
 Rental_ID      0
Vehicle_ID     0
Location_ID    0
Start_Date     0
End_Date       0
dtype: int64

Dados ausentes em Locations:
 Location_ID      0
Location_Name    0
City             0
Country          0
dtype: int64


no missing data was found

## 2.2 Duplicate Check

In [6]:
# 2. Verificar registros duplicados
print("\nRegistros duplicados em Vehicle_Rentals:",
      vehicle_rentals_df.duplicated().sum())
print("Registros duplicados em Locations:", locations_df.duplicated().sum())


Registros duplicados em Vehicle_Rentals: 0
Registros duplicados em Locations: 0


no duplicated rows also. Let's take a look into date consistence in the df `vehicle_rentals_df`

## 2.3 Date Consistency

In [7]:
invalid_dates = vehicle_rentals_df[vehicle_rentals_df["Start_Date"]
                                   > vehicle_rentals_df["End_Date"]]
print("\nNumber of rows with invalid date:", len(invalid_dates))


Number of rows with invalid date: 365


In [8]:
invalid_dates

,Rental_ID,Vehicle_ID,Location_ID,Start_Date,End_Date
16,17,1,1,29/06/2023,01/07/2023
26,27,1,1,31/07/2023,01/08/2023
33,34,2,1,29/05/2023,01/06/2023
39,40,2,1,27/06/2023,03/07/2023
53,54,3,1,28/05/2023,03/06/2023
...,...,...,...,...,...
3413,3414,161,10,30/06/2023,01/07/2023
3420,3421,161,10,29/07/2023,01/08/2023
3429,3430,162,10,31/05/2023,04/06/2023
3436,3437,162,10,26/06/2023,02/07/2023


We already now that our date is in the DD/MM/YYYY format and pandas expect MM/DD/YYYY format. Now let's check for inconsistency in the values of day, month and year:

In [9]:
# Function to check if we have date before 2023
def check_year_before_2023(date_str):
    _, _, year = date_str.split("/")
    return int(year) < 2023


before_2023_start = vehicle_rentals_df[vehicle_rentals_df['Start_Date'].apply(
    check_year_before_2023)]
before_2023_end = vehicle_rentals_df[vehicle_rentals_df['End_Date'].apply(
    check_year_before_2023)]

before_2023_dates = pd.concat(
    [before_2023_start, before_2023_end]).drop_duplicates()

before_2023_dates[['Rental_ID', 'Start_Date', 'End_Date']]

,Rental_ID,Start_Date,End_Date
510,511,30/07/202,31/07/2023
543,544,10/06/203,12/06/2023
1423,1424,10/05/202,12/05/2023
1693,1694,03/06/023,07/06/2023
1813,1814,23/07/223,31/07/2023
1953,1954,23/07/202,27/07/2023
2429,2430,27/07/023,28/07/2023
2432,2433,05/05/202,08/05/2023
2911,2912,02/05/203,07/05/2023
3132,3133,22/05/223,26/05/2023


We don't have date before 2023, jut these wrong year inputs '202', '0203'.. as we can't access the source to understand what may happend, we'll replace these rows with '2023' in the year.

Now let's take a look to inconsistency in day and month.

In [10]:
def check_day_month_validity(date_str):
    day, month, _ = date_str.split("/")
    return 1 <= int(day) <= 31 and 1 <= int(month) <= 12


# Filtrar registros com dias ou meses inválidos em Start_Date e End_Date
invalid_day_month_start = vehicle_rentals_df[~vehicle_rentals_df['Start_Date'].apply(
    check_day_month_validity)]
invalid_day_month_end = vehicle_rentals_df[~vehicle_rentals_df['End_Date'].apply(
    check_day_month_validity)]

invalid_day_month_dates = pd.concat(
    [invalid_day_month_start, invalid_day_month_end]).drop_duplicates()

invalid_day_month_dates[['Rental_ID', 'Start_Date', 'End_Date']]

,Rental_ID,Start_Date,End_Date
189,190,15/13/2023,20/07/2023
212,213,32/07/2023,29/07/2023
1049,1050,45/05/2023,14/05/2023
1551,1552,05/15/2023,07/05/2023
2091,2092,13/20/2023,16/06/2023
2143,2144,21/07/2023,28/15/2023
2397,2398,01/06/2023,35/06/2023


In this case, we identify 7 inconsistent values for days or months, to fix this we should have access to the source to understand better what may happend, how we can't infer as we did for year, we're going to remove these rows from our data.

In [11]:
def replace_year_with_2023(date_str):
    if check_year_before_2023(date_str):
        day, month, _ = date_str.split("/")
        return f"{day}/{month}/2023"
    return date_str


vehicle_rentals_df['Start_Date'] = vehicle_rentals_df['Start_Date'].apply(
    replace_year_with_2023)
vehicle_rentals_df['End_Date'] = vehicle_rentals_df['End_Date'].apply(
    replace_year_with_2023)

In [12]:
valid_start_dates = vehicle_rentals_df['Start_Date'].apply(
    check_day_month_validity)
valid_end_dates = vehicle_rentals_df['End_Date'].apply(
    check_day_month_validity)

# Filtrar o DataFrame usando as máscaras
vehicle_rentals_df = vehicle_rentals_df[valid_start_dates & valid_end_dates]

Now that we've treated all inconsistencies, let's transform the fields to datetime

In [13]:
vehicle_rentals_df['Start_Date'] = pd.to_datetime(
    vehicle_rentals_df['Start_Date'], errors='coerce', format='%d/%m/%Y')
vehicle_rentals_df['End_Date'] = pd.to_datetime(
    vehicle_rentals_df['End_Date'], errors='coerce', format='%d/%m/%Y')

In [14]:
vehicle_rentals_df, vehicle_rentals_df.dtypes

(      Rental_ID  Vehicle_ID  Location_ID Start_Date   End_Date
 0             1           1            1 2023-05-02 2023-05-04
 1             2           1            1 2023-05-05 2023-05-07
 2             3           1            1 2023-05-08 2023-05-09
 3             4           1            1 2023-05-10 2023-05-13
 4             5           1            1 2023-05-14 2023-05-15
 ...         ...         ...          ...        ...        ...
 3439       3440         162           10 2023-07-12 2023-07-16
 3440       3441         162           10 2023-07-17 2023-07-19
 3441       3442         162           10 2023-07-20 2023-07-23
 3442       3443         162           10 2023-07-24 2023-07-26
 3443       3444         162           10 2023-07-27 2023-08-01
 
 [3437 rows x 5 columns],
 Rental_ID               int64
 Vehicle_ID              int64
 Location_ID             int64
 Start_Date     datetime64[ns]
 End_Date       datetime64[ns]
 dtype: object)

In [15]:
vehicle_rentals_df['Start_Date'].isna().sum(
), vehicle_rentals_df['End_Date'].isna().sum()

(0, 0)

Now that we've dealed with the inconsistency cases into date columns, let's validate the consistency in IDs fields to make sure that we have correpondency between tables and they're uniques:

## 2.4 IDs Consistency

In [16]:
# Check if all Location_ID in Vehicle_Rentals.csv are in Locations.csv
assert set(vehicle_rentals_df['Location_ID']).issubset(
    set(locations_df['Location_ID']))

In [17]:
# Check if Rental_ID in Vehicle_Rentals.csv are unique
assert vehicle_rentals_df['Rental_ID'].nunique() == len(
    vehicle_rentals_df), "Existem Rental_IDs duplicados em Vehicle_Rentals.csv"

# Check if Location_ID in Locations.csv are unique
assert locations_df['Location_ID'].nunique() == len(
    locations_df), "Existem Location_IDs duplicados em Locations.csv"

No problems with the IDs.

In [18]:
mask = (vehicle_rentals_df['End_Date'] -
        vehicle_rentals_df['Start_Date']) > pd.Timedelta(days=5*365)
outliers = vehicle_rentals_df[mask]
print(outliers)

Empty DataFrame
Columns: [Rental_ID, Vehicle_ID, Location_ID, Start_Date, End_Date]
Index: []


In [19]:
locations_df[['Location_Name', 'City', 'Country']].drop_duplicates()

,Location_Name,City,Country
0,Jardim Paulista I,São Paulo,Brazil
1,Jardim Paulista II,São Paulo,Brazil
2,Vila Olimpia,São Paulo,Brazil
3,Batel I,Curitiba,Brazil
4,Batel II,Curitiba,Brazil
5,Alto da XV,Curitiba,Brazil
6,Asa Norte I,Brasília,Brazil
7,Asa Norte II,Brasília,Brazil
8,Asa Sul,Brasília,Brazil
9,Copacabana,Rio de Janeiro,Brazil


In [28]:
def plot_boxplot(data, x_title, y_title, title):
    trace = go.Box(
        y=data,
        boxpoints='outliers',
        marker=dict(
            color=color2,
            outliercolor=color2
        ),
        line=dict(color=color2),
        fillcolor=color1,
        whiskerwidth=0.2,
        boxmean=True
    )

    layout = gen_layout(title, x_title, y_title, 50, 400)
    layout['plot_bgcolor'] = color3
    layout['paper_bgcolor'] = color3

    fig = go.Figure(data=[trace], layout=layout)
    fig.show()


plot_boxplot(vehicle_rentals_df['Rental_Duration'], 'Boxplot',
             'Rental Duration(Days)', 'Boxplot of Rental Duration')

We don't have any outlier, our rental duration distributed between 2 and 10 days.